In [1]:
!pip install spacy && python -m spacy download en
!pip install sentence-transformers

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 2.3MB 13.4MB/s 
     |████████████████████████████████| 1.2MB 34.9MB/s 
     |████████████████████████████████| 3.3MB 39.3MB/s 
     |████████████████████████████████| 901kB 30.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=ac702b03ec54e0d5e699a41fa6d06228477ad8177288410198fc22ed00c3ba4f
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [2]:
from pathlib import Path
import os, re, io
import pandas as pd
import numpy as np
import requests

from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.metrics.pairwise import cosine_similarity

import random
import pickle

from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch.utils.data import DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
folder="/content/drive/MyDrive/Sem 3-2 Online/Question Answering System/Final/Code/1-1 Training"

In [5]:
qA_model=pickle.load(open(folder+'/qA_model.sav', 'rb'))
qQ_model=pickle.load(open(folder+'/qQ_model.sav', 'rb'))

In [6]:
class TextPreprocessor():
    def __init__(self, data_df, column_name=None):
        self.data_df = data_df  
        if not column_name and type(colum_name) == str:
            raise Exception("column name is mandatory. Make sure type is string format")
        self.column = column_name
        self.convert_lowercase()    
        self.applied_stopword = False
        self.processed_column_name = f"processed_{self.column}"
        
    def convert_lowercase(self):
        self.data_df.fillna('',inplace=True)
        self.data_df = self.data_df.apply(lambda column: column.astype(str).str.lower(), axis=0)    

    def remove_question_no(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'^\d+[.]',' ', row))    
        
    def remove_symbols(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'[^A-Za-z0-9\s]', ' ', row))    

    def process(self):
        self.remove_question_no()
        self.remove_symbols()  
        return self.data_df

In [7]:
def similarity_score(u, v):
 return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [8]:
def rank_results(qQ_scores, qA_scores):
  qA_top10=sorted(list(enumerate(qA_scores)), key=lambda x: x[1])[-10:]
  index= [i for i,score in qA_top10]
  qQ_top10 = [(i,qQ_scores[i]) for i in index] 
  ranked=sorted(qQ_top10, key=lambda x: x[1])
  index_ranked=[i for (i,score) in ranked][::-1][:5]
  faqID_ranked=list(QA_df.FaqID.iloc[index_ranked])
  print(faqID_ranked)
  return faqID_ranked

In [9]:
def find_relevance_score(faqID_ranked, query_grpID):
  relevance_df=pd.read_csv(folder+'/faqir_relevance.csv')
  val_list=[]
  for faqID in faqID_ranked:
    if (relevance_df['Val'][(relevance_df['FaqID'] == faqID) & (relevance_df['GrpID'] == query_grpID)].empty ==False):
      val_list.append(relevance_df['Val'][(relevance_df['FaqID'] == faqID) & (relevance_df['GrpID'] == query_grpID)].values[0])
    else:
      val_list.append(5)
  return val_list

In [10]:
def performance_metrics(relevance_list):
  p5=0.0
  rr=0.0
  ap=0.0

  if(1 in relevance_list):
    p5=relevance_list.count(1)/5.0
    rr=1.0/(relevance_list.index(1)+1)

    ct=0
    b=[]
    for i in range(len(relevance_list)):
      if relevance_list[i] ==1:
        ct=ct+1
      b.append(ct/(float)(i+1))
    index_1=[i for i in range(len(relevance_list)) if relevance_list[i] == 1] 
    ap=sum([b[i] for i in index_1])/ct

  
  return p5,ap,rr

In [11]:
processed_QA_df=pd.read_csv(folder+'/processed_QA_df.csv',index_col=0)
QA_df = pd.read_csv(folder+'/faqir_faq.csv',header=[0],index_col=0)

In [12]:
processed_QA_df['Q_encoding']=[qQ_model.encode(Q) for Q in processed_QA_df['FaqQuestion']]
processed_QA_df['A_encoding']=[qA_model.encode(A) for A in processed_QA_df['FaqAnswer']]

In [13]:
text_preprocessor_query = TextPreprocessor(pd.read_csv(folder+'/faqir_query.csv', index_col=0), column_name=['Query'])
test_data=text_preprocessor_query.process()
test_data = test_data.astype({'Query_GrpID':int}) 

In [14]:
m_p5=[]
m_ap=[]
m_rr=[]
for i in test_data.index:
  q=test_data['Query'][i]
  print(q)
  print(test_data['Query_GrpID'][i])
  qQ_query_encode=qQ_model.encode(q)
  qA_query_encode=qA_model.encode(q)
  qQ_scores=[similarity_score(qQ_query_encode,Q_encode) for Q_encode in processed_QA_df['Q_encoding']]
  qA_scores=[similarity_score(qA_query_encode,A_encode) for A_encode in processed_QA_df['A_encoding']]
  relevance_list=find_relevance_score(rank_results(qQ_scores, qA_scores), test_data['Query_GrpID'][i])
  print(relevance_list)
  p5,ap,rr= performance_metrics(relevance_list)
  print(p5,ap,rr)
  m_p5.append(p5)
  m_ap.append(ap)
  m_rr.append(rr)

Streaming output truncated to the last 5000 lines.
what s the clutch for in car with manual transmission 
9
[22198, 96258, 43333, 106948, 121060]
[2, 5, 4, 4, 4]
0.0 0.0 0.0
i need a reference to a guidebook on driving manual transmission cars 
9
[26971, 74115, 122782, 103356, 62676]
[5, 4, 5, 5, 5]
0.0 0.0 0.0
is there a driving school where you can drive a car with a stick 
9
[140966, 21010, 21145, 141067, 70902]
[4, 4, 5, 4, 5]
0.0 0.0 0.0
how do europeans drive cars that do not shift gears automatically 
9
[21699, 106948, 1550, 46060, 40844]
[4, 4, 5, 5, 5]
0.0 0.0 0.0
driving a car with a stick 
9
[140966, 102132, 34639, 101338, 16555]
[4, 5, 5, 5, 5]
0.0 0.0 0.0
i got a used car with manual transmission for my 18th birthday  it s a nightmare  how can you drive this thing 
9
[33128, 96944, 76887, 141903, 99667]
[4, 4, 5, 5, 3]
0.0 0.0 0.0
is there something else to keep an eye on except for the gear shifting 
9
[12282, 105433, 22198, 95453, 75348]
[4, 5, 2, 5, 4]
0.0 0.0 0.0
how d

In [15]:
print('Mean Precision @ 5: '+ str(np.mean(m_p5)))
print('Mean Average Precision: '+ str(np.mean(m_ap)))
print('Mean Reciprocal Rank: '+ str(np.mean(m_rr)))

Mean Precision @ 5: 0.14825628548256287
Mean Average Precision: 0.29957871496800936
Mean Reciprocal Rank: 0.3105974587726412
